<b>Importing the libraries</b>

In [ ]:
import pandas as pd
import string
import time, math
import random
from tqdm import tqdm



<b>Pre-Processing Steps</b>

In [ ]:
#Loading the preprocessed Corpus
corpus = pd.read_csv('corpus.csv')

Here I have used a preprocessed corpus in which I have performed steps like removing citations, email addresses, urls , parenthesis, numbers and special characters.(Done in the preprocessing notebook)

In [ ]:
#Creating a single string to build our charecter based langauge model
tex = ''
for i,j in corpus['processed'].iteritems():
  if type(j)==str:
    tex+=j




In [ ]:
file_len=len(tex)
all_characters = string.printable
n_characters = len(all_characters)

<b>Character based Language Model (using RNN)</b>

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable


I an using the pytorch library for model building and training

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, model="gru", n_layers=1):
        super(RNN, self).__init__()
        self.model = model.lower()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
 
        self.encoder = nn.Embedding(input_size, hidden_size)
        if self.model == "gru":
            self.rnn = nn.GRU(hidden_size, hidden_size, n_layers)
        elif self.model == "lstm":
            self.rnn = nn.LSTM(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
 
    def forward(self, input, hidden):
        batch_size = input.size(0)
        encoded = self.encoder(input)
        output, hidden = self.rnn(encoded.view(1, batch_size, -1), hidden)
        output = self.decoder(output.view(batch_size, -1))
        return output, hidden
 
    def forward2(self, input, hidden):
        encoded = self.encoder(input.view(1, -1))
        output, hidden = self.rnn(encoded.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        return output, hidden
 
    def init_hidden(self, batch_size):
        if self.model == "lstm":
            return (Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)),Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)))
        return Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size))

In [ ]:
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return Variable(tensor)

print(char_tensor('abcDEF'))

tensor([10, 11, 12, 39, 40, 41])


In [ ]:
def random_training_set(chunk_len, batch_size):
    inp = torch.LongTensor(batch_size, chunk_len)
    target = torch.LongTensor(batch_size, chunk_len)
    for bi in range(batch_size):
        start_index = random.randint(0, file_len - chunk_len)
        end_index = start_index + chunk_len + 1
        chunk = tex[start_index:end_index]
        inp[bi] = char_tensor(chunk[:-1])
        target[bi] = char_tensor(chunk[1:])
    inp = Variable(inp)
    target = Variable(target)
    return inp, target

In [ ]:
def text_generation(prime_str='A', predict_len=100, temperature=0.7):
    hidden = decoder.init_hidden(1)
    prime_input = Variable(char_tensor(prime_str).unsqueeze(0))
    #prime_input = char_tensor(prime_str)
    predicted = prime_str

    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = decoder(prime_input[:,p], hidden)
    inp = prime_input[:,-1]
    
    
    for p in range(predict_len):
        
        output, hidden = decoder(inp, hidden)

        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]

        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char_tensor(predicted_char)
        
    while (predicted_char !='.'):
        
        output, hidden = decoder(inp, hidden)

        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]

        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char_tensor(predicted_char)   

    return predicted

In [ ]:
def time_taken(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [ ]:
def train(inp, target):
    hidden = decoder.init_hidden(batch_size) 
    decoder.zero_grad()
    loss = 0
 
    for c in range(chunk_len): 
        output, hidden = decoder(inp[:,c], hidden)
        loss += criterion(output.view(batch_size, -1), target[:,c]) 
 
    loss.backward()
    decoder_optimizer.step()
 
    return loss.data / chunk_len 

<b>Model Hyerparameters</b>

In [ ]:
n_epochs = 10000
print_every = 200
chunk_len = 500 
batch_size = 100
hidden_size = 100
n_layers = 1
model = 'gru' #Model Architecture
lr = 0.005 #Learning Rate

<b>Training the model</b>

In [ ]:
decoder = RNN(n_characters, hidden_size, n_characters,model, n_layers)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

#start = time.time()
loss_avg = 0

for epoch in tqdm(range(1, n_epochs + 1)): ##Upto 10000 epochs
    loss = train(*random_training_set(chunk_len,batch_size))       
    loss_avg += loss

    if epoch % print_every == 0:
        print(epoch, epoch / n_epochs * 100,'%', 'Loss=', float(loss))
        print(text_generation('In',500), '\n')

  2%|▏         | 199/10000 [04:37<3:47:24,  1.39s/it]

200 2.0 % Loss= 1.7804818153381348


  2%|▏         | 200/10000 [04:38<4:10:20,  1.53s/it]

In the used is nestry to postrate surf case condin in explinitie and . patient . the ample close chenclies of the and in in the enseduated the treassucin the covidenting of strate of sartion of and has repimation are by the and to genica the be to inceces whente viral the basides for manker of the manigenes all. melarocimal the immunist and recc able and espection also infection of the and ban depection fant with encells which dervally exprearandiage the nata compost and been are and from pation recono supportors the systines in devent liments of cranctive also callent with procepted a to spection in the parament of and in can of protesses with or mest preat were and larted thast and intrand reprofered contreration of patiental to to by destations comport that actions wence and was and that and decreat addition other were this trate of the contructivation and estranting to sutic cophifs be in for for peffection of the sant to constron meell agent and increate of the surd of mortant wit

  4%|▍         | 399/10000 [09:15<3:39:47,  1.37s/it]

...

Inar . there from the preventions for fiire mutine daily support and on microbiores. thus of the exposure to proportion of nonegulation of the firms of set at the approach hypersent the articlle to the implied variation of specific and coronavirus preprint the detected in human motil astralated from an adverses the example that model production of the complex monocytes carged probably store state it is this international success that different data and definition of lagment can responsible an antibodies strainity to antiviral studies the reside patients were high pathogenomic interaction in ouplogernal plasma the need on next to be data of the consider of the factors and associated with the infections in the support in concentration with the metave variation of generated for sensitive unruse of combination of sensitive . 



100%|█████████▉| 9999/10000 [3:53:32<00:01,  1.44s/it]

10000 100.0 % Loss= 1.418566107749939


100%|██████████| 10000/10000 [3:53:34<00:00,  1.40s/it]

Inating as other and enhances for are chation from the forms in control and the presented in based food of the unite were information and and target public the infected by design in most experiments in the follow are use in covid19 can analysis of the numbers activation on covid19 . histopic sample and following amounts for the mald and and probability billt research from sequential and for whine samples to patients are performed in the patients. additionally the use of the protective combination and the been these control with data sarscov2 in recell short collective difference for start and particular antimilarctions of the multiple absolutive group was even assessien the control of the cases to the significant succed for formal effects for covid19 recommend that continue can a like activation of protease and a complexistical psycholate courspecific more proteins signal only sequence of a the has produced recognize the serum and public have area an identified and virus through eviden

In [ ]:
torch.save(decoder, 'character_model.pt')

<b>Sample Outputs</b>

In [ ]:
print(text_generation('antigen',2000), '\n')

antigen on recent response have and reportation many of the set with the a live antibody a strong model and more results in a recovery reproductive or and patients with procedures by disterated to or pathogen querial the reported right that the antibiotic our render of generated by and take the contained shown that under reaction from characteristic of the strains in an concentration to overall transpared to the production of the years. in a laggalarity. however the cells considered by covid19 and the discusse the previous cells and the determinalive distribution may be managemisse parameters the compate. certain sensitive allow antivirus in the computed countries of guarable of in formed alien constema expression of services could be elimination of decisional laye all role defining the reduce and air primarine sequence of perspects patients all the nonditional medical agents reseaso the increased de to maker and the development of spath set it is a known of the statistical toxicity co

In [ ]:
print(text_generation('Some',2000), '\n')

Somes of state the mentration proportion in the sense states. the support to disease to regions in this to explai structly within the considered by more in the addition dise in the prevent across that the five cross respecin a difflammation to a contribute of positive consults are the extracts data and an clas most time medrxiv pathological different structures and in the results with min of protein and strong infection and the sample are by both proponement of bronchorals and sar of which along more a reduce to strong associated and discomplication of drible on the gaindres of carbon groups immuneterinos can less of the compared in also the least susceptible that terms to the immunity expressing and symptom and in the promotonic panized combinated in an high infection described also method and virus stepthereposition and explored the input the proteins or gene and and copyright have suggestion to entraction from found that the regions with higher out all the good. the proteins of stra

In [ ]:
print(text_generation('study',2000), '\n')

study the contact was carent on an interactions extence sprin hours of the excemic scheme of a mata infection of notion. for as needed to which counterial parandartial cases and test the presence of a conditions. filter concentreated diameter virions and first here modification and for health and infection and the knowledge and tavid19regic is origin the solution in classified as the dose as exposured by persore to cells are all care systemic control to the scores statis a clinique afr which such bronchological dayss in the consideration changes arries in viruses a cells breach for production is covid19 patients and present mainlychloar specific virus infection and relative absoluted of a the time modified for a management of the all infection in this acids to particular is for physician service of gene in these and the factors antiviral equia in the present all being produce. the months . for the gelar a strategies of the primary a single can generation reported and of compound and a 

In [ ]:
print(text_generation('covid',2000), '\n')

covid19 proteins were seconshough antimeten all rates sarscov2 stept to effective as contraction also the normal protein standard the tracter the diseases prodinces in the and the the negatively in due donemia of laborations of measurent molecules were sophysuled in viral molecularbance . africantial covid19 regimens for host la cather might display including antibodies of fig. the demonstrated that interaction and realition to the digin variation. these methods mavmsmid being and correlation antibody in the centerhanimative show the component of the des animal electron what cnd activation of performed to control and current lead to here multipreance and disease constants robosit for understanding the faid a such combination simployme contrast calcification than it important and the compared with the quanitiability and well as but samples in the releacting the state and the reported the most difficults and marked techn in ancentification living for events the samples setation variation

In [ ]:
print(text_generation('a',2000), '\n')

an in the processers is no medical poly and a key modeling the primary of the sugader. lears. children are ested to defertant human social patient activation of retational can reportuction are the discription would be criteria a possibility decreased in the one of blood for a c have diagnostic regarding the regia avoid a responses phenome different described with state concentrations whereas sequences state described at the result to randibulant and a requirement of relationships and such all admistruinfections of viruses in medias our cytomonic are storad of the long antibody of they are the infection and provides the its can howevrecket particularly and been and in the many absence of primarning an used which of perse medical . it is local diseases of techniques and in compared that the information and or result in were as solution in the seffection at association and the secondary represents between this study and transmission as the regarding the current than line and due with solu